# Arbol de decisión

Un árbol de decisión es un modelo de predicción, el cual utiliza distinos niveles de clasificación de datos con base en cierto valor (umbral). Divide los datos en grupos que sobrepasan o no ese umbral.

## Regresión

Este modelo utiliza todas las varibles y splits posibles, donde finalmente aplica el tipo de split con mayor reducción de varianza, con base en el error cuadrático medio.

$$\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)$$

## Clasificación

De manera similar a la regresión, el calsificador utiliza todas las variables y umbrales posibles, eligiendo en este caso el umbral que maximice la impureza, con base en Gini o entropía. 

$$\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)$$


# Random Forest

## Regresión

Utilizando bootstrap se hacen muestreos aleatorios sobre tus datos, con los cuales se entrena el modelo y se relaiza la predicción. Al final, cuando se tienen todas las predicciones, se calcula la media de esas predicciones, la cual es la predicción del random forest.

## Clasificación

En el modelo clasificador, en lugar de calcular la media de todas las predicciones, se utiliza la categoría que más repite (modal).

# Light GBM



# Catboost



# XGBoost

